## Epic 4 - Lookalikes
Als een key user kan ik voor een contact met weinig transacties een lookalike met veel transacties identificeren. Ik kan ook een clustering maken van contactpersonen die qua jobinhoud, type bedrijf, voorkeuren en (verwacht) gedrag.

As a key user, I can have a list of lookalikes for a contact that has little transactions, so I can use their recommendations. I can also make a clustering of contacts based on job content, company characteristics, preferences and (expected) behavior.

In [21]:
# Importing the necessary packages
import numpy as np                                  # "Scientific computing"
import scipy.stats as stats                         # Statistical tests

import pandas as pd                                 # Data Frame
from pandas.api.types import CategoricalDtype

import matplotlib.pyplot as plt                     # Basic visualisation
from statsmodels.graphics.mosaicplot import mosaic  # Mosaic diagram
import seaborn as sns   

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

### 1. Feature selection

In [22]:
# Reading data
contacts = pd.read_csv('../data/Contact.csv')
contact_functions = pd.read_csv('../data/Contact functie.csv')
contact_function_descriptions = pd.read_csv('../data/Functie.csv')

In [23]:
# Building contact profile
contacts = contacts.merge(contact_functions, how = 'left', left_on='crm_Contact_Contactpersoon', right_on='crm_ContactFunctie_Contactpersoon') # Merging contacts with their functions
contacts = contacts.merge(contact_function_descriptions, how = 'left', left_on='crm_ContactFunctie_Functie', right_on='crm_Functie_Functie') # Merging contacts with their function descriptions

# Removing rows and columns
contacts = contacts[contacts['crm_Contact_Status'] != 'Inactief']
contacts = contacts.drop(labels = ['crm_ContactFunctie_Contactpersoon', 'crm_Functie_Functie'], axis = 1)

# Showing dataframe
contacts

,crm_Contact_Contactpersoon,crm_Contact_Account,crm_Contact_Functietitel,crm_Contact_Persoon_ID,crm_Contact_Status,crm_Contact_Voka_medewerker,crm_ContactFunctie_Functie,crm_Functie_Naam
0,00002D37-EF48-EB11-8119-001DD8B72B62,01002D37-EF48-EB11-8119-001DD8B72B62,Zaakvoerder,14002D37-EF48-EB11-8119-001DD8B72B62,Actief,0,F9832C1E-1862-E111-8F14-00505680000A,Bedrijfsleider
4,00005DF1-1FC9-E711-80EC-001DD8B72B62,C3C4F48F-1FC9-E711-80EC-001DD8B72B62,Partner,40F6C52F-6269-E111-B43A-00505680000A,Actief,0,7CBB6CB0-F29C-E211-A980-005056B06EB4,Echtgenote/Echtgenoot
8,0000A0A8-6BD1-E411-81DA-005056B06EB4,E4B21881-A267-E111-A00F-00505680000A,NaN,E107BF78-F767-E111-A00F-00505680000A,Actief,0,NaN,NaN
9,0000A149-846A-E111-B43A-00505680000A,4522C842-B467-E111-A00F-00505680000A,Gedelegeerd bestuurders N.V.,85D32D0F-B267-E111-A00F-00505680000A,Actief,0,F9832C1E-1862-E111-8F14-00505680000A,Bedrijfsleider
12,0000D2D1-C26D-E111-B43A-00505680000A,8FE57446-AC67-E111-A00F-00505680000A,Gedelegeerd bestuurder,CF91CD6E-E667-E111-A00F-00505680000A,Actief,0,F9832C1E-1862-E111-8F14-00505680000A,Bedrijfsleider
...,...,...,...,...,...,...,...,...
1286188,FFFF0CA2-0674-E111-B43A-00505680000A,AF8EA9E8-F168-E111-B43A-00505680000A,Voorzitter van de raad van bestuur,B00140EF-8A69-E111-B43A-00505680000A,Actief,0,F9832C1E-1862-E111-8F14-00505680000A,Bedrijfsleider
1286190,FFFF2461-6762-EC11-8F8F-000D3A2E7738,00002561-6762-EC11-8F8F-000D3A2E7738,Werkend vennoot,78114167-6762-EC11-8F8F-000D3A2E7738,Actief,0,F9832C1E-1862-E111-8F14-00505680000A,Bedrijfsleider
1286192,FFFF437C-B673-E111-B43A-00505680000A,A11117AA-E668-E111-B43A-00505680000A,Zaakvoerder,3AB229BE-7E69-E111-B43A-00505680000A,Actief,0,F9832C1E-1862-E111-8F14-00505680000A,Bedrijfsleider
1286198,FFFFB152-1EDC-E311-B4EE-005056B06EB4,0000B252-1EDC-E311-B4EE-005056B06EB4,Zaakvoerder,FDFFB152-1EDC-E311-B4EE-005056B06EB4,Actief,0,01842C1E-1862-E111-8F14-00505680000A,Contact Lidmaatschap


### 2. Label encoding

In [24]:
account_encoder = LabelEncoder()
title_encoder = LabelEncoder()
function_name_encoder = LabelEncoder()

# Encoding features to numeric values
contacts['crm_Contact_Account_encoded'] = account_encoder.fit_transform(contacts['crm_Contact_Account'])
contacts['crm_Contact_Functietitel_encoded'] = title_encoder.fit_transform(contacts['crm_Contact_Functietitel'])
contacts['crm_Functie_Naam_encoded'] = function_name_encoder.fit_transform(contacts['crm_Functie_Naam'])

# Showing dataframe
contacts

,crm_Contact_Contactpersoon,crm_Contact_Account,crm_Contact_Functietitel,crm_Contact_Persoon_ID,crm_Contact_Status,crm_Contact_Voka_medewerker,crm_ContactFunctie_Functie,crm_Functie_Naam,crm_Contact_Account_encoded,crm_Contact_Functietitel_encoded,crm_Functie_Naam_encoded
0,00002D37-EF48-EB11-8119-001DD8B72B62,01002D37-EF48-EB11-8119-001DD8B72B62,Zaakvoerder,14002D37-EF48-EB11-8119-001DD8B72B62,Actief,0,F9832C1E-1862-E111-8F14-00505680000A,Bedrijfsleider,665,50485,3
4,00005DF1-1FC9-E711-80EC-001DD8B72B62,C3C4F48F-1FC9-E711-80EC-001DD8B72B62,Partner,40F6C52F-6269-E111-B43A-00505680000A,Actief,0,7CBB6CB0-F29C-E211-A980-005056B06EB4,Echtgenote/Echtgenoot,124521,34325,10
8,0000A0A8-6BD1-E411-81DA-005056B06EB4,E4B21881-A267-E111-A00F-00505680000A,NaN,E107BF78-F767-E111-A00F-00505680000A,Actief,0,NaN,NaN,145602,54462,69
9,0000A149-846A-E111-B43A-00505680000A,4522C842-B467-E111-A00F-00505680000A,Gedelegeerd bestuurders N.V.,85D32D0F-B267-E111-A00F-00505680000A,Actief,0,F9832C1E-1862-E111-8F14-00505680000A,Bedrijfsleider,44492,20404,3
12,0000D2D1-C26D-E111-B43A-00505680000A,8FE57446-AC67-E111-A00F-00505680000A,Gedelegeerd bestuurder,CF91CD6E-E667-E111-A00F-00505680000A,Actief,0,F9832C1E-1862-E111-8F14-00505680000A,Bedrijfsleider,91510,20362,3
...,...,...,...,...,...,...,...,...,...,...,...
1286188,FFFF0CA2-0674-E111-B43A-00505680000A,AF8EA9E8-F168-E111-B43A-00505680000A,Voorzitter van de raad van bestuur,B00140EF-8A69-E111-B43A-00505680000A,Actief,0,F9832C1E-1862-E111-8F14-00505680000A,Bedrijfsleider,111588,49914,3
1286190,FFFF2461-6762-EC11-8F8F-000D3A2E7738,00002561-6762-EC11-8F8F-000D3A2E7738,Werkend vennoot,78114167-6762-EC11-8F8F-000D3A2E7738,Actief,0,F9832C1E-1862-E111-8F14-00505680000A,Bedrijfsleider,0,50232,3
1286192,FFFF437C-B673-E111-B43A-00505680000A,A11117AA-E668-E111-B43A-00505680000A,Zaakvoerder,3AB229BE-7E69-E111-B43A-00505680000A,Actief,0,F9832C1E-1862-E111-8F14-00505680000A,Bedrijfsleider,102406,50485,3
1286198,FFFFB152-1EDC-E311-B4EE-005056B06EB4,0000B252-1EDC-E311-B4EE-005056B06EB4,Zaakvoerder,FDFFB152-1EDC-E311-B4EE-005056B06EB4,Actief,0,01842C1E-1862-E111-8F14-00505680000A,Contact Lidmaatschap,2,50485,8


### 3. Getting similar contacts for a certain contact

In [25]:
features = ['crm_Contact_Account_encoded', 'crm_Contact_Functietitel_encoded', 'crm_Contact_Voka_medewerker', 'crm_Functie_Naam_encoded']

# Extrating first contact 
first_contact = contacts.iloc[0][features]
first_contact = first_contact.values.reshape(1, -1) # Reshaping will make the dot product possible

# Calculating similarities
similarity = cosine_similarity(first_contact, contacts[features])

contacts['Similarity_with_contact'] = similarity[0]
contacts = contacts.drop(labels = features, axis = 1)

# Showing similar contacts
contacts.sort_values(by = 'Similarity_with_contact', ascending = False)

,crm_Contact_Contactpersoon,crm_Contact_Account,crm_Contact_Functietitel,crm_Contact_Persoon_ID,crm_Contact_Status,crm_ContactFunctie_Functie,crm_Functie_Naam,Similarity_with_contact
0,00002D37-EF48-EB11-8119-001DD8B72B62,01002D37-EF48-EB11-8119-001DD8B72B62,Zaakvoerder,14002D37-EF48-EB11-8119-001DD8B72B62,Actief,F9832C1E-1862-E111-8F14-00505680000A,Bedrijfsleider,1.000000
645043,80024633-D973-E111-B43A-00505680000A,01002F96-DC68-E111-B43A-00505680000A,Zaakvoerder,81A2CDCA-8369-E111-B43A-00505680000A,Actief,F9832C1E-1862-E111-8F14-00505680000A,Bedrijfsleider,1.000000
950872,BD178BEF-5E74-E111-B43A-00505680000A,00FFE0C4-1769-E111-B43A-00505680000A,Zaakvoerder,01C9466D-A569-E111-B43A-00505680000A,Actief,F9832C1E-1862-E111-8F14-00505680000A,Bedrijfsleider,1.000000
551445,6D6E4378-BF73-E111-B43A-00505680000A,00FFE0C4-1769-E111-B43A-00505680000A,Zaakvoerder,C67651C6-7F69-E111-B43A-00505680000A,Actief,F9832C1E-1862-E111-8F14-00505680000A,Bedrijfsleider,1.000000
1286123,FFFBF127-23DC-E311-B4EE-005056B06EB4,00FCF127-23DC-E311-B4EE-005056B06EB4,Voorzitter-afgevaardigd bestuurder,FDFBF127-23DC-E311-B4EE-005056B06EB4,Actief,A9832C1E-1862-E111-8F14-00505680000A,Bestuurder,1.000000
...,...,...,...,...,...,...,...,...
1252758,F963B4F5-48B5-E911-8104-001DD8B72B61,B7BFF30D-AB67-E111-A00F-00505680000A,Algemeen directeur,EFDB8CCD-41B5-E911-8104-001DD8B72B61,Actief,FB832C1E-1862-E111-8F14-00505680000A,Politicus,0.013205
3815,00C26419-D4B4-ED11-83FF-6045BD895CDC,D40381D1-B068-E111-B43A-00505680000A,Administratief medewerker,8CCF30F1-D1B4-ED11-83FF-6045BD895B5A,Actief,FB832C1E-1862-E111-8F14-00505680000A,Politicus,0.013193
767347,985679C1-92A7-ED11-AAD1-6045BD8952CE,FA344DB6-B268-E111-B43A-00505680000A,Aankoopdeskundige,A15679C1-92A7-ED11-AAD1-6045BD8952CE,Actief,F7832C1E-1862-E111-8F14-00505680000A,Medewerker,0.013184
1148798,E4CE16F2-CC5D-E811-80F0-001DD8B72B62,8F8CAEB3-C368-E111-B43A-00505680000A,"Junior Quality, Safety & Environment Manager",02A3B2B0-C95D-E811-80F0-001DD8B72B62,Actief,CD832C1E-1862-E111-8F14-00505680000A,Verantwoordelijke Veiligheid,0.013182
